**런타임 실행전 할 것**
<br/> Daphnia_dataset.zip 업로드

#학습 전 처리

In [2]:
!git clone https://github.com/ultralytics/yolov5 #clone repo
%cd yolov5
%pip install -qr requirements.txt #개발 환경 다운로드 받기


Cloning into 'yolov5'...
remote: Enumerating objects: 12228, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 12228 (delta 15), reused 20 (delta 8), pack-reused 12195
Receiving objects: 100% (12228/12228), 11.98 MiB | 12.66 MiB/s, done.
Resolving deltas: 100% (8459/8459), done.
/content/yolov5
     |████████████████████████████████| 596 kB 29.4 MB/s 


이 단계에서 daphnia.yaml을 yolov5/data안에 넣을것

In [3]:
import torch
from IPython.display import Image, clear_output #to display images

clear_output()

In [4]:
if torch.cuda.is_available():    #CPU 쓰는지 GPU 쓰는지 판단
  DEVICE=torch.device('cuda')
  print('Device name:',torch.cuda.get_device_name(0))
else:
  DEVICE=torch.device('cpu')
  
print('using PyTorch version:',torch.__version__,'Device:',DEVICE)

Device name: Tesla T4
using PyTorch version: 1.11.0+cu113 Device: cuda


사진과 주석 압축 풀기
 - train test data 분류 완료(90:10)


In [5]:
!unzip -q ../Daphnia_dataset.zip -d../

In [9]:
#glob 모듈 사용 image들을 list로 제작 후 list를 txt 파일로 생성성

from glob import glob
 
# 이미지들의 주소 리스트로 만들어줌
train_img_list = glob('../Daphnia_dataset/images/train/*.jpg')
valid_img_list = glob('../Daphnia_dataset/images/val/*.jpg')

#txt 파일로 생성
f=open("../Daphnia_dataset/train.txt",'w')
f.close()
f=open("../Daphnia_dataset/val.txt",'w')
f.close()
 
# 리스트를 txt파일로 생성
with open('../Daphnia_dataset/train.txt', 'w') as f:
      	f.write('\n'.join(train_img_list) + '\n')
    
with open('../Daphnia_dataset/val.txt', 'w') as f:
      	f.write('\n'.join(valid_img_list) + '\n')

In [29]:
# 데이터 경로 재설정

import yaml
 
with open('./data/daphnia.yaml', 'r') as f:
  	data = yaml.load(f,Loader=yaml.FullLoader)
    
#print(data)
 
data['train'] = '../Daphnia_dataset/train.txt'
data['valid'] = '../Daphnia_dataset/val.txt'
 
with open('../yolov5/data/daphnia.yaml', 'w') as f:
	yaml.dump(data, f)

In [30]:
#데이터 덮어쓰기(경로 등) 잘 되었는지 확인인
print(data)

{'path': '../Daphnia_dataset', 'train': '../Daphnia_dataset/train.txt', 'val': 'val.txt', 'nc': 1, 'names': ['daphnia'], 'valid': '../Daphnia_dataset/val.txt'}


#YOLOv5s 모델을 통한 학습과정

In [32]:

!python train.py --img 640 --batch 2 --epochs 60 --data daphnia.yaml --weights yolov5s.pt --nosave --cache

train: weights=yolov5s.pt, cfg=, data=daphnia.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=60, batch_size=2, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-250-g6adc53b Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.

# YOLOv5l를 사용해서 더욱 정확히 학습


차이점
<br/> epochs 60->120으로 증가
<br/> 모듈의 경우 YOLOv5l이 YOLOv5s보다 크기가 커 정확도가 올라간다.

In [36]:
!python train.py --img 640 --batch 2 --epochs 120 --data daphnia.yaml --weights yolov5l.pt --nosave --cache

train: weights=yolov5l.pt, cfg=, data=daphnia.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=120, batch_size=2, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-250-g6adc53b Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0

#학습결과확인

학습 이후 yolov5/runs/train 보면 실제 주석 데이터랑 모델이 예측한 데이터 있으니 확인해볼것!